In [4]:
# Import libraries
import pandas as pd
from datetime import datetime
import pytz

# Keras imports
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

# Preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

# Saving model
from keras.models import load_model

imputer = SimpleImputer()
MMS = MinMaxScaler()
model = load_model('../Model.h5')

,Unnamed: 0,Date,Home,Away,Home Odds,Away Odds
0,0,2021-07-11 23:31:38.166432-04:00,American League,National League,1.97,1.952


In [ ]:
# Today and yesterday date EST
tz = pytz.timezone('America/New_York')
today = datetime.now(tz)
year = today.year

# Get future games
games = pd.read_csv('./auto_files/future_games.csv')
games['Home'] = 'Arizona Diamondbacks'
games['Visitor'] = 'Atlanta Braves'

In [13]:
# Read stats and only keep necessary columns

stats = pd.read_csv('../Scraping/MLB All Stats.csv')

# Find a year's columns
cols = stats.columns
str_cols = [str(col) for col in cols]
sum('2021' in s for s in str_cols)
remove_dupes = [i[5:] for i in str_cols]
one_year_cols = []
[one_year_cols.append(x) for x in remove_dupes if x not in one_year_cols]
one_year_cols.pop(0)
final_cols = [str(year) + ' ' + i for i in one_year_cols]
final_cols.insert(0, 'Tm')
stats = pd.read_csv('../Scraping/MLB All Stats.csv', usecols=final_cols)

# Make home and away stat dfs
home_stats = stats.add_prefix('H ')
vis_stats = stats.add_prefix('V ')

# Rename Team column
home_stats.rename(columns={'H Tm': 'Home'}, inplace=True)
vis_stats.rename(columns={'V Tm': 'Visitor'}, inplace=True)

# Merge Stats and games
merged = games.merge(home_stats, on='Home')
merged = merged.merge(vis_stats, on='Visitor')
x = merged.loc[:,'H 2021 #Bat':]

In [44]:
# Get previous data so we can fit_transform on it then transform on predictions x

data_current = pd.read_csv('../Scraping/Current Stats and Games.csv', parse_dates=['Date'])

y = data_current['Home Win']
x_current = data_current.loc[:,'H  #Bat':]

# Split columns with hyphens
# Create function to use on both current and prev stats
def fit_transform(x, y):
    
    x = x.astype(str)
    cols_to_delim = []
    for col in x.columns:
        result = x[col].str.contains(pat='\d-\d')
        if result.any():
            cols_to_delim.append(col)

    for col in cols_to_delim:
            x[[col + '1', col + '2']] = x[col].str.split('-', expand=True)
            del x[col]

    x = x.astype(float)
    
    # Scale and Normalise
    x = imputer.fit_transform(x, y)
    x = MMS.fit_transform(x)
    return x

x_current = fit_transform(x_current, y)

In [47]:
# Transform predictive columns

def transform(x):
    
    x = x.astype(str)
    cols_to_delim = []
    for col in x.columns:
        result = x[col].str.contains(pat='\d-\d')
        if result.any():
            cols_to_delim.append(col)

    for col in cols_to_delim:
            x[[col + '1', col + '2']] = x[col].str.split('-', expand=True)
            del x[col]

    x = x.astype(float)
    
    # Scale and Normalise
    x = imputer.transform(x)
    x = MMS.transform(x)
    return x

x_trans = transform(x)

In [57]:
preds = model.predict(x_trans)

In [63]:
preds

array([[0.5478382]], dtype=float32)

In [65]:
games['Predictions'] = preds

In [71]:
NE_input = games[['Date', 'Home', 'Visitor', 'Home Odds', 'Away Odds', 'Predictions']]

In [76]:
NE_input.to_csv('./auto_files/ne_input.csv', index=False)

In [77]:
games

,Unnamed: 0,Date,Home,Away,Home Odds,Away Odds,Visitor,Predictions
0,0,2021-07-11 23:31:38.166432-04:00,Arizona Diamondbacks,National League,1.97,1.952,Atlanta Braves,0.547838
